# 0. パッケージのインポート

In [3]:
import numpy as np
from scipy.optimize import minimize
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import warnings
warnings.filterwarnings('ignore')

# 1. パラメータの設定

- 総濃度の探索
    - `c_max`: 探索上限値   
    - `c_step`: １軸あたり探索ステップ

In [37]:
c_max = 1.0e-4
c_step = 50

# 2. 平衡定数の指定

平衡定数は **別ファイルで指定** する。`acid.dis`には、上から順に平衡定数 $K_{\text a1}, K_{\text a2},\dots$ を記録せよ。

$$
\begin{aligned}
    \mathrm H_{m\phantom{{}-0}}\mathrm A^{\phantom{-}} &\stackrel{K_{\text a1}}{\rightleftarrows} \mathrm H^+ + \mathrm H_{m-1}\mathrm A^{\phantom{1}-}\\
    \mathrm H_{m-1}\mathrm A^- &\stackrel{K_{\text a2}}{\rightleftarrows} \mathrm H^+ + \mathrm H_{m-2}\mathrm A^{2-}\\
    &\hspace{0.75em}\vdots
\end{aligned}
$$

`base.dis`には、上から順に平衡定数 $K_{\text b1}, K_{\text b2},\dots$ を記録せよ。

$$
\begin{aligned}
    {\mathrm B(\mathrm{OH})_{n-1}}^{\phantom{1}+} + \mathrm{H_2O} &\stackrel{K_{\text b1}}{\rightleftarrows} {\mathrm B(\mathrm{OH})_{n\phantom{{}-0}}}^{\phantom{+}} + \mathrm H^+\\ 
    {\mathrm B(\mathrm{OH})_{n-2}}^{2+} + \mathrm{H_2O} &\stackrel{K_{\text b2}}{\rightleftarrows} {\mathrm B(\mathrm{OH})_{n-1}}^+ + \mathrm H^+\\ 
    &\hspace{0.75em}\vdots
\end{aligned}
$$

上の式で定義される塩基の平衡定数 $K_{\text bi}$ がわからなければ、以下の変換公式が有効である。
$$ K_{\text bi}=\frac{K_{\text w}}{K_{\text bi}'},\quad {\mathrm B(\mathrm{OH})_{n-i+1}}^{(i-1)+}\stackrel{K_{\text bi}'}{\rightleftarrows} {\mathrm B(\mathrm{OH})_{n-i}}^{i+}+\mathrm{OH}^- $$

# 3. 実行

以上のセルの実行が正常にできれば、次のセルを実行してpHの3Dプロットが得られる。
`out.dat`に実行結果が記録され、セルの下にグラフが表示される。

In [ ]:
kw = 1.008e-14

def p_inv(x):
    return 10**(-x)

def read_acidbase(file_acid: str, file_base: str):
    global kw
    
    acid_arr = np.insert(np.genfromtxt(file_acid, dtype=float), 0, 1.)
    base_arr = np.insert(np.genfromtxt(file_base, dtype=float), 0, 1.)

    if type(acid_arr) != np.ndarray:
        acid_arr = np.array([acid_arr])
    if type(base_arr) != np.ndarray:
        base_arr = np.array([base_arr])
    return acid_arr, base_arr

def equation(x, ca: float, cb: float):
    global acid_arr, base_arr, kw

    m, n = len(acid_arr), len(base_arr)

    nmr_a, dnm_a, nmr_b, dnm_b = 0., 0., 0., 0.
    for i in range(m+1):
        nmr_a += i * np.prod(acid_arr[0:i+1]) / p_inv(x) ** i
        dnm_a +=     np.prod(acid_arr[0:i+1]) / p_inv(x) ** i
    for j in range(n+1):
        nmr_b += j * p_inv(x) ** j / np.prod(base_arr[0:j+1])
        dnm_b +=     p_inv(x) ** j / np.prod(base_arr[0:j+1])

    a = ca * nmr_a / dnm_a - cb * nmr_b / dnm_b
    b = -kw
    ans = p_inv(x) - (a + np.sqrt(a ** 2 - 4 * b)) / 2
    return np.log10(np.abs(ans))

def solve(ca: float, cb: float):
    global equation, acid_arr, base_arr, kw

    x_guess = minimize(equation, 7, args=(ca, cb), method='Powell', tol=1e-12).x[0]
    return x_guess

def run(c_max: float, step: int):
    global kw
    X, Y, Z = [], [], []

    x_arr = np.linspace(0., c_max, step, endpoint=False)
    y_arr = np.linspace(0., c_max, step, endpoint=False)

    for x in x_arr:
        X_sub, Y_sub, Z_sub = [], [], []
        for y in y_arr:
            z = solve(x, y)
            X_sub.append(x)
            Y_sub.append(y)
            Z_sub.append(z)
        X.append(X_sub)
        Y.append(Y_sub)
        Z.append(Z_sub)
    
    return np.array(X), np.array(Y), np.array(Z)

def plot(X: list, Y: list, Z: list):
    fig = plt.figure(figsize=(12,12))
    ax = fig.add_subplot(111, projection='3d')

    ax.plot_surface(X, Y, Z, cmap='gist_rainbow', vmin=0, vmax=14)

    ax.set_xlabel(r'Total Concentration of Acid $C_{\text{A}}$')
    ax.set_ylabel(r'Total Concentration of Base $C_{\text{B}}$')
    ax.set_zlabel(r'pH')

    ax.set_zlim(0,14)

    plt.show()

acid_arr, base_arr = read_acidbase('acid.dis', 'base.dis')
X, Y, Z = run(c_max, c_step)

with open('out.dat', 'w') as f:
    for i, Z_sub in enumerate(Z):
        for j, z in enumerate(Z_sub):
            f.write(f'{X[i][j]}\t{Y[i][j]}\t{z}\n')
plot(X, Y, Z)

## 原理

電気中性の法則から得る $[\mathrm H^+]$ の多項式
$$ [\mathrm H^+]^2 - \left(C_{\text A}\cdot\frac{\displaystyle\sum_{i=0}^m\,i\cdot\frac{K_{\text a1}\cdots K_{\text ai}}{[\mathrm H^+]^i}}{\displaystyle\sum_{i=0}^m\phantom{\,i\cdot{}}\frac{K_{\text a1}\cdots K_{\text ai}}{[\mathrm H^+]^i}}-C_{\text B}\cdot\frac{\displaystyle\sum_{i=0}^n\,i\cdot\frac{[\mathrm H^+]^i}{K_{\text b1}\cdots K_{\text bi}}}{\displaystyle\sum_{i=0}^n\phantom{\,i\cdot{}}\frac{[\mathrm H^+]^i}{K_{\text b1}\cdots K_{\text bi}}}\right)[\mathrm H^+] - K_{\text w} = 0 $$
を、形式的に $[\mathrm H^+]$ の2次方程式とみなし解の公式を得る。
$$ [\mathrm H^+] = \frac{a+\sqrt{a^2-4b}}2,\quad a=\left(C_{\text A}\cdot\frac{\displaystyle\sum_{i=0}^m\,i\cdot\frac{K_{\text a1}\cdots K_{\text ai}}{[\mathrm H^+]^i}}{\displaystyle\sum_{i=0}^m\phantom{\,i\cdot{}}\frac{K_{\text a1}\cdots K_{\text ai}}{[\mathrm H^+]^i}}-C_{\text B}\cdot\frac{\displaystyle\sum_{i=0}^n\,i\cdot\frac{[\mathrm H^+]^i}{K_{\text b1}\cdots K_{\text bi}}}{\displaystyle\sum_{i=0}^n\phantom{\,i\cdot{}}\frac{[\mathrm H^+]^i}{K_{\text b1}\cdots K_{\text bi}}}\right),\quad b=-K_{\text w} $$
$ [\mathrm H^+] = \frac{a+\sqrt{a^2-4b}}2 $ は右辺が $[\mathrm H^+]$ に依るため、一種の自己無撞着方程式になっており、
$$ f([\mathrm H^+]) = \log\left|[\mathrm H^+] - \frac{a+\sqrt{a^2-4b}}2\right| $$
が最小になる $[\mathrm H^+]$ を`Powell`法で探索して求める。

総濃度 $C_{\text A}, C_{\text B}$ は滴下する溶液に依って場合分けされる。
- 塩基を酸で滴定するなら、滴下体積を$v$として、$$ C_{\text A} = \frac{v}{v+V_{\text B}^{(0)}},\quad C_{\text B} = \frac{V_{\text B}^{(0)}}{v+V_{\text B}^{(0)}} $$
- 酸を塩基で滴定するなら、滴下体積を$v$として、$$ C_{\text A} = \frac{V_{\text A}^{(0)}}{V_{\text A}^{(0)}+v},\quad C_{\text B} = \frac{v}{V_{\text A}^{(0)}+v} $$